In [4]:
import importlib
import pickle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

import funciones_auxiliares_hito_2 as aux2

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (20, 10)

# Modelacion

## Carga de datos de entrenamiento y validacion

- En este caso, se optimizaran los modelos utilizando PCA en las variables independientes y utilizando SMOTE en los vectores objetivos

In [23]:
train_problema_1, train_problema_2 = aux2.preprocesar_todo_prueba_2_machine_learning(
    "input/2009_1perc.csv"
)
test_problema_1, test_problema_2 = aux2.preprocesar_todo_prueba_2_machine_learning(
    "input/2010_1perc.csv"
)

#####################################################
X_train_problema_1 = train_problema_1.drop(columns="arstmade")
X_test_problema_1 = test_problema_1.drop(columns="arstmade")

dif_1_problema_1, dif_2_problema_1 = aux2.obtener_difs_columnas(
    X_train_problema_1, X_test_problema_1
)

X_train_problema_1 = X_train_problema_1.drop(columns=dif_1_problema_1)
X_test_problema_1 = X_test_problema_1.drop(columns=dif_2_problema_1)

y_train_problema_1 = train_problema_1["arstmade"]
y_test_problema_1 = test_problema_1["arstmade"]
######################################################
#####################################################
X_train_problema_2 = train_problema_2.drop(columns="arstmade")
X_test_problema_2 = test_problema_2.drop(columns="arstmade")

dif_1_problema_2, dif_2_problema_2 = aux2.obtener_difs_columnas(
    X_train_problema_2, X_test_problema_2
)

X_train_problema_2 = X_train_problema_2.drop(columns=dif_1_problema_2)
X_test_problema_2 = X_test_problema_2.drop(columns=dif_2_problema_2)

y_train_problema_2 = train_problema_2["arstmade"]
y_test_problema_2 = test_problema_2["arstmade"]
######################################################

['perobs']
[]


## Optimizacion

### PCA

In [24]:
from sklearn.decomposition import PCA

pca_problema_1 = PCA(n_components=10)

X_train_problema_1 = pca_problema_1.fit_transform(X_train_problema_1)
X_test_problema_1 = pca_problema_1.transform(X_test_problema_1)

pca_problema_2 = PCA(n_components=10)

X_train_problema_2 = pca_problema_2.fit_transform(X_train_problema_2)
X_test_problema_2 = pca_problema_2.transform(X_test_problema_2)


### SMOTE

In [26]:
from imblearn.over_sampling import SMOTE

oversampler_problema_1 = SMOTE(random_state=1, sampling_strategy="minority")

X_train_problema_1, y_train_problema_1 = oversampler_problema_1.fit_resample(
    X_train_problema_1, y_train_problema_1
)

oversampler_problema_2 = SMOTE(random_state=1, sampling_strategy="minority")
X_train_problema_2, y_train_problema_2 = oversampler_problema_2.fit_resample(
    X_train_problema_2, y_train_problema_2
)

In [31]:
y_train_problema_1.value_counts()

0    4355
1    4355
Name: arstmade, dtype: int64

In [32]:
y_train_problema_2.value_counts()

0    4355
1    4355
Name: arstmade, dtype: int64

# Reporte comparativo de desempenos

- En el Hito 2 se reportaron los desempenos de cada uno de los modelos ya entrenados. Esto se realizo
tanto para el conjunto de entrenamiento y el conjunto de validacion. En la siguiente tabla se
recopila a modo de resumen los resultados.

- En este caso se reportara tambien el f1-score de la clase 1 para cada problema. Esto, ya que es
la clase de mayor interes para cada problema

## Problema 1 - Prediccion de Arresto

|Modelo|Entrenamiento Accuracy|Validacion F1 Clase 1|Validacion Accuracy|Validacion ROC|
|------|----------------------|---------------------|-------------------|--------------|
|Logistico|0.97|0.75|0.97|0.82|
|Naive Bayes|0.96|0.69|0.96|0.86|
|Gradient Boosting|0.97|0.95|0.99|0.96|




## Problema 2 - Detencion Violenta

|Modelo|Entrenamiento Accuracy|Validacion F1 Clase 1|Validacion Accuracy|Validacion ROC|
|------|----------------------|---------------------|-------------------|--------------|
|Logistico|0.77|0.26|0.77|0.57|
|Naive Bayes|0.76|0.39|0.76|0.61|
|Gradient Boosting|0.76|0.65|0.87|0.74|
|Arbol de Clasificacion|0.76|0.26|0.76|0.56|
|Random Forest|0.77|0.37|0.81|0.62|

# Exportacion de Mejores Modelos

Teniendo en cuenta los resultados anteriores, los mejores modelos son el modelo de regresion
logistica y naive bayes. Por lo tanto, ambos seran exportados para validacion externa.

In [4]:
modelos_a_exportar = {
    "javier_rojas-modelo-1.pickle": modelos["modelo_gradient_boosting_problema_1"],
    "javier_rojas-modelo-2.pickle": modelos["modelo_gradient_boosting_problema_2"],
}


In [5]:
for archivo, modelo in modelos_a_exportar.items():
    with open(archivo, "wb") as file:
        pickle.dump(modelo, file)
